In [1]:
cd /home/chenxupeng/projects/deepshape/

/home/chenxupeng/projects/deepshape


In [2]:
import h5py

/home/chenxupeng/apps/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import numpy as np
import gc
import datetime
import argparse, sys, os, errno
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
import keras as K
import sys
sys.path.append('bin')
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from unet_128_model_row_column import *
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # so the IDs match nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.preprocessing.image import ImageDataGenerator
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.9
#set_session(tf.Session(config=config))

Using TensorFlow backend.


In [58]:
def get_data(count):
    with h5py.File('data/new/Spitale_2015_invivo/train/train_CDS_stride16_'+str(count)) as f:
        x_train = f['train_images'][:]
        y_train = f['y_train'][:]
    return x_train,y_train

In [ ]:
for i in range(16):
    with h5py.File('data/new/Spitale_2015_invivo/train/train_all_winsize128_stride16_'+str(i)) as f:
        f.create_dataset('y_train',data=y_train[i*10000:(i+1)*10000])
for i in [17]:
    with h5py.File('data/new/Spitale_2015_invivo/train/train_all_winsize128_stride16_'+str(i)) as f:
        f.create_dataset('y_train',data=y_train[i*10000:])

In [12]:
data.shape

(10000, 128, 128, 16)

In [22]:
def data_generator(data, labels, batch_size):
    batches = (data.shape[0] + batch_size - 1)//batch_size
    while(True):
        for i in range(batches):
            X = data[i*batch_size : (i+1)*batch_size]
            Y = labels[i*batch_size : (i+1)*batch_size]
            yield (X, Y)

In [40]:
model = UNET_128()
optim = Adam()
model.compile(optimizer=optim, loss=binary_crossentropy_with_nan, metrics=[binary_accuracy_with_nan,mse_with_nan])
#model.summary()

In [66]:
def Model(X_train, Y_train,batch_size=64,num_epochs=100):
    model_checkpoint = ModelCheckpoint('output/unet_cdsstride16_4.2.hdf5', 
                        monitor='binary_accuracy_with_nan', save_best_only=True)
    callbacks=[model_checkpoint,EarlyStopping(monitor='CrossEntropyLoss', 
    patience=10, verbose=0),TensorBoard(log_dir='deepshape/output/tensorboard/unet/log_dir')]
    
    model.fit_generator(generator = data_generator(X_train, Y_train, batch_size),
                        steps_per_epoch = (data.shape[0] + batch_size - 1) // batch_size,
                        epochs = num_epochs,verbose = 1,callbacks = callbacks)
    
    model.save('output/unet_cdsstride16_4.2.hdf5')

In [65]:
for i in range(8):
    x_train,y_train = get_data(i)
    Model(x_train,y_train,64,100)
    gc.collect()

TypeError: fit_generator() got an unexpected keyword argument 'validation_split'

In [55]:
77*7*100/3600.

14.972222222222221